Ironhack_gambling.ipynb


In [31]:
import pymysql

cnx = pymysql.connect(user='root', password='Malcomx1',
                      host='localhost', database='sakila')
if cnx.open:
    print("Connection open")
else:
    print("Connection is not successfully open")

Connection open


In [32]:
# the object which will interact with the database
cursor = cnx.cursor()

In [33]:
import pandas as pd

# Specify the file path for the Excel file
file_path = r'dataset\SQL Test Data.xlsx'

# Load the Excel file into a pandas DataFrame
try:
    # Read the Excel file
    sql_test_data = pd.ExcelFile(file_path)

    # Display all sheet names
    print("Available sheets:", sql_test_data.sheet_names)

    # Load each sheet into a dictionary of DataFrames
    ironhack_gambling_df = {sheet: sql_test_data.parse(sheet) for sheet in sql_test_data.sheet_names}

    # Display all columns in every sheet
    for sheet, df in ironhack_gambling_df.items():
        print(f"\nSheet: {sheet}")
        print(f"Columns: {df.columns.tolist()}")
        print(df.head())

    # Display the first few rows of each sheet
    for sheet, df in ironhack_gambling_df.items():
        print(f"\nSheet: {sheet}")
        print(df.head())

except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found. Please check the path.")
except Exception as e:
    print(f"An error occurred: {str(e)}")


Available sheets: ['Account', 'Customer', 'Betting', 'Product', 'Student_School']

Sheet: Account
Columns: ['AccountNo', 'CustId', 'AccountLocation', 'CurrencyCode', 'DailyDepositLimit', 'StakeScale', 'SourceProd']
    AccountNo   CustId AccountLocation CurrencyCode  DailyDepositLimit   
0  00357DG     3531845             GIB          GBP                  0  \
1  00497XG     4188499             GIB          GBP                  0   
2  00692VS     4704925             GIB          USD                  0   
3  00775SM     2815836             GIB          USD                  0   
4  00C017       889782             GIB          GBP               1500   

   StakeScale SourceProd  
0        1.00         GM  
1        1.00         SB  
2        2.00         SB  
3        1.00         SB  
4        0.41         XX  

Sheet: Customer
Columns: ['CustId', 'AccountLocation', 'Title', 'FirstName', 'LastName', 'CreateDate', 'CountryCode', 'Language', 'Status', 'DateOfBirth', 'Contact', 'CustomerGr

In [34]:
# Sheet to Table Mapping
sheet_to_table_mapping = {
    'Account': 'Account',
    'Customer': 'Customer',
    'Betting': 'Betting',
    'Product': 'Product',
    'Student_School': 'Student_School'
}

In [ ]:
import pymysql
import numpy as np

# Connect to MySQL
try:
    cnx = pymysql.connect(user='root', password='Malcomx1',
                        host='localhost', database='SQLTestData')
    if cnx.open:
        print("Connection open")
    else:
        print("Connection is not successfully open")
    
    # the object which will interact with the database
    cursor = cnx.cursor()

    # Mapping of cleaned column names to MySQL table columns
    table_column_mapping = {
    'Account': ['AccountNo', 'CustId', 'AccountLocation', 'CurrencyCode', 'DailyDepositLimit', 'StakeScale', 'SourceProd'],
    'Customer': ['CustId', 'AccountLocation', 'Title', 'FirstName', 'LastName', 'CreateDate', 'CountryCode', 'Language',
                 'Status', 'DateOfBirth', 'Contact', 'CustomerGroup'],
    'Betting': ['AccountNo', 'BetDate', 'ClassId', 'CategoryId', 'Source', 'BetCount', 'Bet_Amt', 'Win_Amt', 'Product',
                'AccountNo2', 'Vegas', 'Sportsbook', 'Games', 'Casino', 'Poker', 'Bingo', 'Adjustments'],
    'Product': ['CLASSID', 'CATEGORYID', 'product', 'sub_product', 'description', 'bet_or_play'],
    'Student_School': ['student_id', 'student_name', 'city', 'school_id', 'GPA', 'school_name', 'school_city']
}

    for sheet, table in sheet_to_table_mapping.items():
        # Load sheet into DataFrame
        df = sql_test_data.parse(sheet)

        # Clean column names
        df.columns = df.columns.str.strip()
        df.columns = df.columns.str.replace('[^a-zA-Z0-9_]', '_', regex=True)
        df.columns = df.columns.str.replace('^Unnamed.*', 'column_', regex=True)

        # Replace NaN values
        df = df.replace({np.nan: None})

        # Drop duplicates for the 'Account' sheet
        if sheet == 'Account':
            df['AccountNo'] = df['AccountNo'].str.strip()
            df = df.drop_duplicates(subset=['AccountNo'])

        # Map DataFrame columns to MySQL table schema
        expected_columns = table_column_mapping[table]
        # df = df[expected_columns]  # Select and reorder columns to match the MySQL table schema

        for col in expected_columns:
            if col not in df.columns:
                df[col] = None  # Add missing column with default value
        df = df[expected_columns]  # Reorder columns to match MySQL schema


        # Create INSERT query
        columns = ', '.join(expected_columns)
        placeholders = ', '.join(['%s'] * len(expected_columns))
        # insert_query = f"INSERT INTO {table} ({columns}) VALUES ({placeholders})"
        insert_query = f"INSERT IGNORE INTO {table} ({columns}) VALUES ({placeholders})" # INSERT IGNORE skips rows with duplicate keys


        # Insert DataFrame into MySQL table
        for _, row in df.iterrows():
            cursor.execute(insert_query, tuple(row))
        
        cnx.commit()
        print(f"Data from sheet '{sheet}' inserted into table '{table}' successfully.")

except pymysql.Error as e:
    print(f"Error: {e}")
finally:
    cursor.close()
    cnx.close()
    print("MySQL connection closed.")


Connection open
Data from sheet 'Account' inserted into table 'Account' successfully.
Data from sheet 'Customer' inserted into table 'Customer' successfully.
Data from sheet 'Betting' inserted into table 'Betting' successfully.
Data from sheet 'Product' inserted into table 'Product' successfully.
Data from sheet 'Student_School' inserted into table 'Student_School' successfully.
MySQL connection closed.
